In [ ]:
%pip install numpy
%pip install pandas
%pip install tensorflow
%pip install numpy
%pip install scikit-learn

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam

In [157]:
batting_df = pd.read_csv('Batting.csv')
as_df = pd.read_csv('AllstarFull.csv')

In [158]:
batting_df = batting_df[batting_df['yearID'] >=1973]
batting_df = batting_df[batting_df['yearID'] <= 2014]
batting_df = batting_df[batting_df['AB'] >= 200]
batting_df.drop(['GIDP', 'SF' ,'SH', 'HBP', 'lgID' ,'teamID', 'stint' ,'CS'], inplace=True, axis=1)
batting_df.dropna(inplace=True)
batting_df.reset_index(inplace=True)
batting_df.drop(['index'], inplace=True, axis=1)
batting_df.head()

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,IBB
0,aaronha01,1973,120,392.0,84.0,118.0,12.0,1.0,40.0,96.0,1.0,68.0,51.0,13.0
1,ageeto01,1973,83,204.0,30.0,48.0,5.0,2.0,8.0,15.0,2.0,16.0,55.0,1.0
2,allendi01,1973,72,250.0,39.0,79.0,20.0,3.0,16.0,41.0,7.0,33.0,51.0,3.0
3,alomasa01,1973,136,470.0,45.0,112.0,7.0,1.0,0.0,28.0,25.0,34.0,44.0,1.0
4,aloufe01,1973,93,280.0,25.0,66.0,12.0,0.0,4.0,27.0,0.0,9.0,25.0,5.0


In [159]:
as_df = as_df[as_df['yearID'] >= 1974]
as_df = as_df[as_df['startingPos'] >1]
as_df.drop(['gameNum', 'gameID', 'teamID', 'lgID' ,'GP', 'startingPos'], inplace=True, axis=1)
as_df.reset_index(inplace=True)
as_df.drop(['index'], inplace=True, axis=1)
as_df.head()

,playerID,yearID
0,munsoth01,1974
1,allendi01,1974
2,carewro01,1974
3,robinbr01,1974
4,campabe01,1974


In [151]:
df = batting_df
df['ASNextYear'] = np.nan

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,IBB,ASNextYear
0,aaronha01,1973,120,392.0,84.0,118.0,12.0,1.0,40.0,96.0,1.0,68.0,51.0,13.0,NaN
1,ageeto01,1973,83,204.0,30.0,48.0,5.0,2.0,8.0,15.0,2.0,16.0,55.0,1.0,NaN
2,allendi01,1973,72,250.0,39.0,79.0,20.0,3.0,16.0,41.0,7.0,33.0,51.0,3.0,NaN
3,alomasa01,1973,136,470.0,45.0,112.0,7.0,1.0,0.0,28.0,25.0,34.0,44.0,1.0,NaN
4,aloufe01,1973,93,280.0,25.0,66.0,12.0,0.0,4.0,27.0,0.0,9.0,25.0,5.0,NaN


In [ ]:
for bIndex,bRow in batting_df.iterrows():
    for aIndex,aRow, in as_df.iterrows():
        if bRow['yearID'] == aRow['yearID'] -1 and bRow['playerID'] == aRow['playerID']:
            print(bRow['playerID'], bRow['yearID'])
            df.at[bIndex, 'ASNextYear']= 1
            as_df.drop(aIndex, inplace=True)
            
df = df.fillna(0)

In [153]:
df.to_csv('CapstoneData.csv', sep=',', index=False, encoding='utf-8')

# Nueral Network

In [16]:
df = pd.read_csv('CapstoneData.csv')
df = df[df['yearID'] >= 2000]
df.head()

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,IBB,ASNextYear
7504,abbotje01,2000,80,215.0,31.0,59.0,15.0,1.0,3.0,29.0,2.0,21.0,38.0,1.0,0.0
7505,abreubo01,2000,154,576.0,103.0,182.0,42.0,10.0,25.0,79.0,28.0,100.0,116.0,9.0,0.0
7506,agbaybe01,2000,119,350.0,59.0,101.0,19.0,1.0,15.0,60.0,5.0,54.0,68.0,2.0,0.0
7507,alfoned01,2000,150,544.0,109.0,176.0,40.0,2.0,25.0,94.0,3.0,95.0,70.0,1.0,0.0
7508,alicelu01,2000,139,540.0,85.0,159.0,25.0,8.0,6.0,63.0,1.0,59.0,75.0,1.0,0.0


In [17]:
features = df[['AB','R','H','2B','3B','HR','RBI','BB','SO']]
label = df[['ASNextYear']]

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.5, random_state = 42)

std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train)
X_test_std = std_scaler.transform(X_test)

norm_scaler = MinMaxScaler()
X_train_norm = norm_scaler.fit_transform(X_train)
X_test_norm = norm_scaler.transform(X_test)

In [18]:
model = Sequential()
model.add(Input(shape=(9,)))
model.add(Dense(256, 'relu'))
model.add(Dense(128, 'relu'))
model.add(Dense(1, 'sigmoid'))

model_std = Sequential()
model_std.add(Input(shape=(9,)))
model_std.add(Dense(256, 'relu'))
model_std.add(Dense(128, 'relu'))
model_std.add(Dense(1, 'sigmoid'))

model_norm = Sequential()
model_norm.add(Input(shape=(9,)))
model_norm.add(Dense(256, 'relu'))
model_norm.add(Dense(128, 'relu'))
model_norm.add(Dense(1, 'sigmoid'))

model.compile(optimizer=Adam(0.001), metrics=['accuracy'], loss = 'binary_crossentropy')
model_std.compile(optimizer=Adam(0.001), metrics=['accuracy'], loss = 'binary_crossentropy')
model_norm.compile(optimizer=Adam(0.001), metrics=['accuracy'], loss = 'binary_crossentropy')

In [19]:
model.fit(X_train, y_train, epochs=3, batch_size = 10)
model_std.fit(X_train_std, y_train, epochs=3, batch_size=10)
model_norm.fit(X_train_norm, y_train, epochs=3, batch_size=10)

Epoch 1/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8988 - loss: 2.8366
Epoch 2/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9389 - loss: 0.4425
Epoch 3/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step - accuracy: 0.9092 - loss: 0.8590
Epoch 1/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9356 - loss: 0.2416
Epoch 2/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9514 - loss: 0.1613
Epoch 3/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9481 - loss: 0.1713
Epoch 1/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9473 - loss: 0.2981
Epoch 2/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9439 - loss: 0.1870
Epoch 3/3
244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9500 - loss: 0.1656


In [20]:
print("UnScaled:", model.evaluate(X_train, y_train), model.evaluate(X_test,y_test))
print("STd:", model.evaluate(X_train_std, y_train), model.evaluate(X_test_std,y_test))
print("norm:", model.evaluate(X_train_norm, y_train), model.evaluate(X_test_norm,y_test))

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9152 - loss: 0.2540
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step - accuracy: 0.9132 - loss: 0.2641
UnScaled: [0.245939239859581, 0.9177969694137573] [0.26705461740493774, 0.9112207293510437]
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.9526 - loss: 0.5622
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.9530 - loss: 0.5647
STd: [0.56208735704422, 0.9515001773834229] [0.5632711052894592, 0.9506781697273254]
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.9526 - loss: 0.5938
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.9530 - loss: 0.5935
norm: [0.5942940711975098, 0.9515001773834229] [0.5947901010513306, 0.9506781697273254]
